# Import librairies

In [1]:
import geopandas as gpd
import numpy as np
import folium
from shapely.geometry import Point, LineString

In [2]:
from modules.connecting_to_bdd import fetch_table_from_trafic_db
from modules.geodata import convert_to_linestring, convert_to_point

# Import dataset

In [3]:
evenements_df = fetch_table_from_trafic_db('events_data')

In [4]:
trafic_df = fetch_table_from_trafic_db('trafic_routier')

In [5]:
trafic_df.head()

,id_technique,id,debit,longueur,taux_occupation,code_couleur,nom_du_troncon,etat_du_trafic,temps_de_parcours,vitesse,geo_point_2d,geometrie,shape_geo,horodatage,type_geo,coordinates_geo
0,772-20250218T205200,772,120,410,1.2,3,Vannes I9,Fluide,62,24,"{47.23414730688424,-1.5808786419612229}","{{-1.583123434893785,47.235185973808406},{-1.5...",LineString,2025-02-18 20:52:00,Point,"{-1.5808786419612229,47.23414730688424}"
1,158-20250218T205200,158,300,364,3.7,3,Le Lasseur P2,Fluide,63,21,"{47.23083661767159,-1.5704468983375661}","{{-1.568589501648445,47.23188191655151},{-1.57...",LineString,2025-02-18 20:52:00,Point,"{-1.5704468983375661,47.23083661767159}"
2,497-20250218T205200,497,180,224,2.0,3,Lauriol I4,Fluide,37,22,"{47.23531301184315,-1.5635261418937887}","{{-1.563539784805616,47.236320687488636},{-1.5...",LineString,2025-02-18 20:52:00,Point,"{-1.5635261418937887,47.23531301184315}"
3,5444-20250218T205200,5444,240,177,2.9,3,Pasteur P1,Fluide,40,16,"{47.21255561611825,-1.579953397082114}","{{-1.581033846171769,47.21286509290865},{-1.57...",LineString,2025-02-18 20:52:00,Point,"{-1.579953397082114,47.21255561611825}"
4,5053-20250218T205200,5053,60,224,0.8,3,Barbusse P2,Fluide,44,18,"{47.22705355840811,-1.5524880999813486}","{{-1.551553912867712,47.22760386550291},{-1.55...",LineString,2025-02-18 20:52:00,Point,"{-1.5524880999813486,47.22705355840811}"


In [6]:
trafic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1554761 entries, 0 to 1554760
Data columns (total 16 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   id_technique       1554761 non-null  object        
 1   id                 1554761 non-null  int64         
 2   debit              1554761 non-null  int64         
 3   longueur           1554761 non-null  int64         
 4   taux_occupation    1554761 non-null  float64       
 5   code_couleur       1554761 non-null  int64         
 6   nom_du_troncon     1554761 non-null  object        
 7   etat_du_trafic     1554761 non-null  object        
 8   temps_de_parcours  1554761 non-null  int64         
 9   vitesse            1554761 non-null  int64         
 10  geo_point_2d       1554761 non-null  object        
 11  geometrie          1554761 non-null  object        
 12  shape_geo          1554761 non-null  object        
 13  horodatage         1554761 

In [7]:
evenements_df.head()

,name,description,startdate,enddate,location_name,address,city,postalcode,location_uid,coordinates_geo,event_id
0,Football : FC Nantes / Paris SG,Match - 29e journée - Ligue 1 - Saison 2024/2025,2025-04-13 17:00:00,2025-04-13 19:00:00,Stade de la Beaujoire Louis Fonteneau,"330 Route de Saint Joseph, Nantes",Nantes,44300,95545941,"{-1.527812, 47.25867}",926afe1f-3bcc-4cc1-a2bd-3597dd36d86d
1,Football : FC Nantes / Toulouse FC,Match - 31e journée - Ligue 1 - Saison 2024/2025,2025-04-27 17:00:00,2025-04-27 19:00:00,Stade de la Beaujoire Louis Fonteneau,"330 Route de Saint Joseph, Nantes",Nantes,44300,95545941,"{-1.527812, 47.25867}",19bb79da-1c20-41a2-a53d-c45bcae5a0ce
2,Blandine Lehout - La Vie de ta mère,Blandine Lehout - La Vie de ta mère,2025-10-01 20:00:00,2025-10-01 22:30:00,Cité internationale des Congrès,"5 Rue de Valmy, Nantes",Nantes,44000,47538434,"{-1.544058, 47.213314}",da51b672-08bc-49e6-bcd6-516090c48e77
3,Alexandre Kominek - Bâtard sensible,Alexandre Kominek - Bâtard sensible,2025-03-01 20:30:00,2025-03-01 22:00:00,Cité internationale des Congrès,"5 Rue de Valmy, Nantes",Nantes,44000,47538434,"{-1.544058, 47.213314}",209c09e0-8d45-4429-ab15-7e1bf979f9c1
4,Comédie Le Clan des divorcées,Comédie Le Clan des divorcées,2025-03-02 15:00:00,2025-03-02 16:40:00,Cité internationale des Congrès,"5 Rue de Valmy, Nantes",Nantes,44000,47538434,"{-1.544058, 47.213314}",2eae861f-2e7f-4ff4-8c77-8f72b719fb23


# Jointure

In [8]:
# Conversion données geo en LineString
trafic_df["geometry"] = trafic_df["geometrie"].apply(convert_to_linestring)
# Conversion données geo en Point
evenements_df["geometry"] = evenements_df["coordinates_geo"].apply(convert_to_point)

In [9]:
# Conversion en GeoDataFrame
trafic_df_gdf = gpd.GeoDataFrame(trafic_df, geometry="geometry", crs="EPSG:3857")
evenements_gdf = gpd.GeoDataFrame(evenements_df, geometry="geometry", crs="EPSG:3857")

In [10]:
# Conserver uniquement les evenements et les tronçons uniques
evenements_gdf_unique = evenements_gdf.drop_duplicates(subset=["coordinates_geo"])
trafic_gdf_unique = trafic_df_gdf.drop_duplicates(subset=["geometrie"])

In [11]:
# Vérification
print(len(evenements_gdf_unique))
print(len(trafic_gdf_unique))

7
849


In [12]:
# Reprojection des GeoDataFrames vers EPSG:2154 (Lambert 93 - unités en mètres)
trafic_gdf_unique = trafic_gdf_unique.to_crs(epsg=2154)
evenements_gdf_unique = evenements_gdf_unique.to_crs(epsg=2154)

# Créer un buffer autour de chaque tronçon de route (500 mètres)
trafic_gdf_unique["buffered_geometry"] = trafic_gdf_unique["geometry"].buffer(500)

# Initialiser les colonnes pour chaque tronçon
trafic_gdf_unique["has_event_near_troncon"] = 0
trafic_gdf_unique["event_location_name"] = np.nan

# Parcours de chaque tronçon pour vérifier si un événement est à moins de 500 mètres
for index, row in trafic_gdf_unique.iterrows():
    # Obtenir le buffer du tronçon 
    troncon_buffer = row["buffered_geometry"]
    
    # Filtrer les lieux d'evenements qui sont à l'intérieur du buffer
    events_nearby = evenements_gdf_unique[evenements_gdf_unique.intersects(troncon_buffer)]
    
    # Vérification si des lieux d'evenements sont proches
    if not events_nearby.empty:
        # Calcul des distances réelles pour chaque lieu d'evenement trouvé à l'intérieur du buffer
        distances = events_nearby.apply(lambda x: x.geometry.distance(row["geometry"]), axis=1)
        distances_in_meters = distances * 100        
        
        # Si des lieux d'evenements sont à moins de 500 mètres, mettre 1 dans la colonne has_event_near_troncon
        closest_event_distance = distances_in_meters.min()  # Distance du plus proche événement
        
        if closest_event_distance <= 0.4:  # Vérification si la distance est bien <= 400m
            trafic_gdf_unique.at[index, "has_event_near_troncon"] = 1
            # Trouver le nom de l'événement le plus proche
            closest_event_idx = distances_in_meters.idxmin()  # L'index de l'événement le plus proche
            trafic_gdf_unique.at[index, "event_location_name"] = events_nearby.loc[closest_event_idx, "location_name"]
            trafic_gdf_unique.at[index, "event_location_id"] = events_nearby.loc[closest_event_idx, "location_uid"]
        else:
            trafic_gdf_unique.at[index, "has_event_near_troncon"] = 0
            trafic_gdf_unique.at[index, "event_location_name"] = np.nan
    else:
        trafic_gdf_unique.at[index, "has_event_near_troncon"] = 0
        trafic_gdf_unique.at[index, "event_location_name"] = np.nan

result_ = trafic_gdf_unique[["id_technique", "nom_du_troncon", "has_event_near_troncon", "event_location_name"]]

<ipython-input-12-fe8e3a155c44>:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Parc des Expositions de la Beaujoire' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  trafic_gdf_unique.at[index, "event_location_name"] = events_nearby.loc[closest_event_idx, "location_name"]


## Vérification en image

In [13]:
# Reprojection vers WGS84 (EPSG:4326) pour affichage sur carte
trafic_gdf_unique = trafic_gdf_unique.to_crs(epsg=4326)
evenements_gdf_unique = evenements_gdf_unique.to_crs(epsg=4326)

In [14]:
trafic_gdf_unique['geometry_4326'] = trafic_gdf_unique['geometrie'].apply(convert_to_linestring)
evenements_gdf_unique['geometry_4326'] = evenements_gdf_unique['coordinates_geo'].apply(convert_to_point)

In [15]:
# Créer une carte centrée sur Nantes
map_nantes = folium.Map(location=[47.2184, -1.5536], zoom_start=13)

# 1. Tronçons SANS événement (bleu)
for idx, row in trafic_gdf_unique[trafic_gdf_unique["has_event_near_troncon"] == 0].iterrows():
    if isinstance(row['geometry_4326'], LineString):
        folium.PolyLine(
            locations=[(coord[1], coord[0]) for coord in row['geometry_4326'].coords],
            color='blue',
            weight=2,
            opacity=0.7,
            tooltip=row.get("nom_du_troncon", "")
        ).add_to(map_nantes)

# 2. Tronçons AVEC événement (vert)
for idx, row in trafic_gdf_unique[trafic_gdf_unique["has_event_near_troncon"] == 1].iterrows():
    if isinstance(row['geometry_4326'], LineString):
        folium.PolyLine(
            locations=[(coord[1], coord[0]) for coord in row['geometry_4326'].coords],
            color='green',
            weight=3.5,
            opacity=1,
            tooltip=f"Événement proche : {row.get('event_location_name', '')}"
        ).add_to(map_nantes)

# 3. Points des événements (rouge)
for idx, event in evenements_gdf_unique.iterrows():
    if isinstance(event['geometry_4326'], Point):
        folium.Marker(
            location=[event['geometry_4326'].y, event['geometry_4326'].x],
            popup=event['location_name'],
            icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(map_nantes)

# Afficher la carte
map_nantes
